In [2]:
import pandas as pd

df = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Pennings/Final_df_interpolated/GA_new_df.csv')

df

,Latitude,Longitude,Sampling_year,LS_year,ndvi_mean_08_10,evi_mean_08_10,nd_swir2_r_mean_08_10,nd_swir2_nir_mean_08_10,nd_g_b_mean_08_10,nd_r_g_mean_08_10,...,spei90d_Min,pdsi_gridmet,spei180d_gridmet,spei1y_gridmet,spei30d_gridmet,spei5y_gridmet,spei90d_gridmet,Yr,lat_r,lon_r
0,31.308449,-81.415129,2013,2013,0.608544,0.185087,0.262307,-0.411777,0.188730,0.020280,...,1.69,-1.323333,0.988333,0.032778,0.311111,-1.355556,0.623333,1/1/2013,31.308449,-81.415129
1,31.308449,-81.415129,2015,2015,0.502339,0.148550,0.240085,-0.298153,0.132298,0.062364,...,1.69,0.580556,0.415556,0.333333,0.433889,-0.873889,0.706111,1/1/2015,31.308449,-81.415129
2,31.308449,-81.415129,2016,2016,0.548571,0.179854,0.294428,-0.302436,0.142160,0.049064,...,1.69,-0.713684,-0.314211,-0.456842,0.080526,-0.234737,-0.184737,1/1/2016,31.308449,-81.415129
3,31.308449,-81.415129,2018,2018,0.497044,0.161636,0.261019,-0.271003,0.122543,0.050631,...,1.69,3.070555,1.047778,0.569444,0.081667,0.937778,0.662222,1/1/2018,31.308449,-81.415129
4,31.308449,-81.415129,2021,2021,0.496204,0.159708,0.267223,-0.263843,0.129631,0.059329,...,2.09,4.330555,1.030000,0.983889,0.826111,1.872222,1.567222,1/1/2021,31.308449,-81.415129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,31.539505,-81.421937,2018,2018,0.539582,0.148651,0.293553,-0.292413,0.173896,0.038968,...,1.69,3.003889,1.262222,0.670556,0.163889,0.208889,0.877778,1/1/2018,31.539505,-81.421937
3077,31.539505,-81.421937,2019,2019,0.453920,0.121159,0.268477,-0.211386,0.140268,0.030576,...,-0.10,-0.112222,-0.970000,-0.271667,-0.491667,0.030714,-0.812222,1/1/2019,31.539505,-81.421937
3078,31.539505,-81.421937,2020,2020,0.572171,0.170051,0.309778,-0.318770,0.322373,0.006610,...,1.69,3.605263,1.011579,1.280000,0.273684,0.736842,0.847368,1/1/2020,31.539505,-81.421937
3079,31.539505,-81.421937,2021,2021,0.566227,0.167103,0.236192,-0.380722,0.192631,0.046198,...,2.09,3.286667,0.833889,0.633333,0.205000,1.288333,1.075556,1/1/2021,31.539505,-81.421937


In [20]:
dfs2 = df[df['Site'] == 2]
dfs2['dayl_Max']

2441   NaN
2442   NaN
2443   NaN
2444   NaN
2445   NaN
        ..
2793   NaN
2794   NaN
2795   NaN
2796   NaN
2797   NaN
Name: dayl_Max, Length: 357, dtype: float64

In [12]:
climate_vars = ['srad_peak', 'dayl_Max', 'spei14d']

df['Year'] = df['LS_year']

donors = (
    df[df['Site'] == 3]
    .dropna(subset=climate_vars)
    .groupby('Year', as_index=False)
    .first()
    [['Year'] + climate_vars]
)

df_filled = df.merge(
    donors,
    on='Year',
    how='left',
    suffixes=('', '_donor')
)

In [14]:
mask = df_filled['Site'] == 2

for v in climate_vars:
    df_filled.loc[mask, v] = (
        df_filled.loc[mask, v]
        .fillna(df_filled.loc[mask, f'{v}_donor'])
    )


In [15]:
df_filled = df_filled.drop(columns=[f'{v}_donor' for v in climate_vars])


In [16]:
print(
    df_filled.loc[df_filled['Site'] == 2, climate_vars]
    .isna()
    .sum()
)
##You want all zeros

srad_peak    16
dayl_Max     16
spei14d      16
dtype: int64


In [21]:
dfs2 = df_filled[df_filled['Site'] == 2]
dfs2['srad_peak']

2441    336.230316
2442    318.082489
2443    332.947388
2444    312.132812
2445    332.019562
           ...    
2793           NaN
2794    349.659027
2795    324.776947
2796    337.880005
2797    332.019562
Name: srad_peak, Length: 357, dtype: float64

In [28]:
dfx = df_filled[['Plant_Biomass','Year','Yr','Site','Latitude','Longitude','evi_max_05_10','slope','elevation',
                 'Percent_flooding','srad_peak','dayl_Max','nd_swir2_nir_mean_05_10',
                 'ndvi_mean_08_10','nd_r_g_max_05_10','spei14d']].dropna()

dfx

,Plant_Biomass,Year,Yr,Site,Latitude,Longitude,evi_max_05_10,slope,elevation,Percent_flooding,srad_peak,dayl_Max,nd_swir2_nir_mean_05_10,ndvi_mean_08_10,nd_r_g_max_05_10,spei14d
0,440.816,2013,1/1/2013,8,31.308449,-81.415129,0.232282,0.693467,1.179146,0.014085,336.610535,50542.17969,-0.404771,0.608544,0.059350,0.119041
1,468.640,2015,1/1/2015,8,31.308449,-81.415129,0.190468,0.693467,1.179146,0.014085,316.011963,50542.17969,-0.317078,0.502339,0.117821,0.104658
2,170.976,2016,1/1/2016,8,31.308449,-81.415129,0.195634,0.693467,1.179146,0.014085,332.383911,50542.17969,-0.336010,0.548571,0.075218,-0.249589
3,140.192,2018,1/1/2018,8,31.308449,-81.415129,0.178204,0.693467,1.179146,0.014085,330.674133,50542.17969,-0.277814,0.497044,0.065727,0.230000
4,435.200,2021,1/1/2021,8,31.308449,-81.415129,0.191262,0.693467,1.179146,0.014085,334.421417,50542.17969,-0.281224,0.496204,0.076267,0.237534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,434.768,2018,1/1/2018,1,31.539505,-81.421937,0.168836,0.308225,1.087664,0.011594,340.065857,50610.12891,-0.262571,0.539582,0.074512,0.224795
3077,324.176,2019,1/1/2019,1,31.539505,-81.421937,0.132574,0.308225,1.087664,0.011594,327.975342,50610.12891,-0.203389,0.453920,0.056916,-0.282676
3078,340.144,2020,1/1/2020,1,31.539505,-81.421937,0.200399,0.308225,1.087664,0.011594,311.941528,50610.12891,-0.307148,0.572171,0.047406,0.063571
3079,246.176,2021,1/1/2021,1,31.539505,-81.421937,0.204570,0.308225,1.087664,0.011594,336.644226,50610.12891,-0.348148,0.566227,0.073944,0.069726


In [29]:
##Exporting coordinates
import os

out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'LOGO_df_interpolated.csv')

# dfx.to_csv(out_csv)